In [1]:
#1
def func(ar):
    ans = {}
    n = len(ar)
    ans["mean"] = sum(ar)/n
    ar = sorted(ar)
    if n%2==0:
        ans["median"] = (ar[n//2 - 1] + ar[n//2]) / 2
    else:
        ans["median"] = ar[n//2]
    counter = {}
    for item in ar:
        counter[item] = counter.get(item, 0) + 1
    max_count = max(counter.values())
    mode = [k for k, v in counter.items() if v == max_count]
    if len(mode)==1:
        ans["mode"] = mode[0]
    else:
        ans["mode"] = mode

    return ans

In [7]:
#2
from typing import List, Dict
def parse_csv(file_content: str) -> List[Dict]:
    lines = file_content.splitlines()
    headers = lines[0].split(',')
    data = []
    for line in lines[1:]:
        values = line.split(',')
        row = {}
        for i in range(len(headers)):
            row[headers[i]] = values[i]
        data.append(row)
    return data

In [12]:
import os
from pathlib import Path
with open(os.path.join( str(Path(os.getcwd()).parents[0]) , 'data/dota_hero_stats.csv'), 'r') as file:
    data = file.read()

In [13]:
file = parse_csv(data)

In [14]:
max_legs = 0
name = ''
for hero in file:
    if int(hero['legs'])>max_legs:
        max_legs = int(hero['legs'])
        name = hero['localized_name']

In [15]:
print("больше всего ног у", name)

больше всего ног у Broodmother


In [ ]:
#3 id, localized_name, name - уникальные значения, поэтому не будем использовать их
# primary_attr не информативен
# будем использовать attack_type, legs и roles
# f1(xi,xj) = 0 if xi[attack_type] == xj[attack_type] else 1
# f2(xi,xj) = |xi[legs] - xj[legs]|/|max(xk[legs]) - min(xq[legs])| по всем k,q
# f3(x1,xj) = количество ролей xi и xj, не пренадлежищих пересечению xi[roles] и xj[roles], деленное на мощность пересечения xi[roles] и xj[roles]
# итоговая мера = f1+f2+f3 (самы дельние герои будут на расстоянии 3, ближние на 0)

In [16]:
min_legs = 10000000
for hero in file:
    if int(hero['legs'])<min_legs:
        min_legs = int(hero['legs'])

In [17]:
min_legs, max_legs

(0, 8)

In [18]:
def f1(xi,xj):
    return xi["attack_type"] != xj["attack_type"]

def f2(xi,xj):
    return abs(int(xi["legs"]) - int(xj["legs"]))/(max_legs-min_legs)

def f3(xi,xj):
    t = 0
    union = (set(xi["roles"]) or set(xj["roles"]))
    intersection = (set(xi["roles"]) and set(xj["roles"]))
    for i in xi["roles"]+xj["roles"]:
        t += i not in intersection
    return t/len(union)
def f(xi,xj):
    return f1(xi,xj) + f2(xi,xj) + f3(xi,xj)

In [19]:
min_d = 3
ans = ""
for i in range(len(file)):
    for j in range(i+1,len(file)):
        if f(file[i], file[j])<min_d:
            min_d = f(file[i], file[j])
            ans = i,j

In [20]:
print("минимальное расстояние", min_d, "между", "\n", file[ans[1]],"\n и \n", file[ans[0]])

минимальное расстояние 0.0 между 
 {'': '3', 'attack_type': 'Melee', 'id': '4', 'legs': '2', 'localized_name': 'Bloodseeker', 'name': 'npc_dota_hero_bloodseeker', 'primary_attr': 'agi', 'roles': '"[\'Carry\''} 
 и 
 {'': '0', 'attack_type': 'Melee', 'id': '1', 'legs': '2', 'localized_name': 'Anti-Mage', 'name': 'npc_dota_hero_antimage', 'primary_attr': 'agi', 'roles': '"[\'Carry\''}
